API 키 불러오고 테스트하기

In [1]:
import os 

from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi' }],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


리뷰 파일 로드하기

In [2]:
import json
with open('./res/yanolja_review.json', 'r') as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '나인트리 인사동점은 처음이었는데 다른데와 달리 인사동스러운 분위기가 있었어요 한국식인데 깔끔하고 우아한 컨셉이라고 해야할지.. 직원분들이 엄청 친절해서 기분좋았고 객실도 너무 깨끗해서 만족스러웠어요ㅎㅎ 조계사 뷰를 받았는데 밤에는 약간 루미나리에 되어있는게, 아침엔 아침풍경이 멋있더라구요.소음도 아침 체크아웃 시간 안팎에 복도 청소소음 정도 뿐이고(이정도는 어디나 있으니까) 아주 푹 잘잤네요!\n어메너티중 핸드워시는 리퀴드 치곤 약간 건조한 편이에요 건조함 예민하신 분들은 비누 따로 챙겨가시거나 하는게 좋을듯 합니다ㅎㅎ',
  'stars': 5,
  'date': '2025.01.08'},
 {'review': '위치도 너무 좋고 시설이 깔끔해서 좋았어요~\n우선 침대랑 샤워실,화장실이 유리로 되어있지 않고 따로 구분되어 있어서 좋았어요~\n저희가 더위를 잘타는데 에어컨도 빵빵해서 중간에 더워서 일어날일 없이 푹 자서 좋았어요😆\n다음에도 인사동 올일 있다면 다시 오고 싶은 곳이에요~',
  'stars': 5,
  'date': '2025.08.19'},
 {'review': '방도 깔끔한 편이고 위치가 워낙 좋아서 만족스러웠어요. 조계사 뷰로 부탁드렸더니 낮은 층수였는데도 경치가 좋았어요. 침구도 푹신하고 책상이나 소파, 화장실도 깔끔했습니다. 조계사, 인사동 구경하기 딱 좋은 위치라 재밌었어요. 단점이라면 방이 덥고 건조하다는 거랑 구조가 조금 복잡한 편이라 길 찾기가 조금 어려울 수 있는 부분인데, 감안하더라도 다음에 또 오고싶을만큼 만족스러웠습니다. 특히 레이트 체크아웃이 가능해서 좋았어요.',
  'stars': 5,
  'date': '2024.12.04'}]

좋은 리뷰, 나쁜 리뷰 구분하고 카운트하기

In [4]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(159, 61)

좋은 리뷰와 나쁜 리뷰 구분 후 저장

In [5]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]') #자연어 처리에 많이 쓰이는 스페셜 토큰
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]') 
reviews_bad[:3]

['[REVIEW_START]조계사 뷰라서 밤에 한국바이브 낭낭하구 완전 좋았어요!! 다만 넷플릭스 등 ott가 안된다는 점과 화장실이 생각보다 낙후됐다는 점이 아쉬웠습니다ㅠㅠ 변기는 짱 좋은데 욕조가!! 살짝 그랬네요!! 하지만 직원분들이 정말 친절하셨고 복도에 얼음정수기도 있다는 점은 정말 좋았어요!![REVIEW_END]',
 '[REVIEW_START]나인트리는 첫 이용인데 한지를 이용한 컨셉의 인테리어와 객실 청결, 프론트까지 다 좋았습니다. 인사동이 바로 앞이라 구경하고 데이트하기에도 좋습니다. 주차비를 받는점이 아쉬웠으나 객실 가격이 합리적인 가격에 책정되었기 때문에 납득 가능했습니다.[REVIEW_END]',
 '[REVIEW_START]직원분들 엄청 친절하고 좋았다\n건물 동선이 길치인 내게 너무 헷갈렸다 \nㅜㅜ\n연박인데 첫날 먼지가 많아서 기침했는데 \n청소 요청한 다음날은 더 쾌적함[REVIEW_END]']

In [ ]:
# 현재 리스트 구조인데, 실제 프롬프트에 넣을 때는 텍스트 구조여야 함.
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:100]

'[REVIEW_START]조계사 뷰라서 밤에 한국바이브 낭낭하구 완전 좋았어요!! 다만 넷플릭스 등 ott가 안된다는 점과 화장실이 생각보다 낙후됐다는 점이 아쉬웠습니다ㅠㅠ 변기는'

전처리 함수 작성

In [14]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/yanolja_review.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for r in review_list:
        review_date_str =r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError): # 날짜 형식이 아닌 경우(n일 전, n시간 전)
            review_date = current_date

        if review_date < date_boundary: # 기준일 이전에 작성된 리뷰는 무시
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]') 
        
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text
    
good, bad = preprocess_reviews()
good[:100]

'[REVIEW_START]위치도 너무 좋고 시설이 깔끔해서 좋았어요~\n우선 침대랑 샤워실,화장실이 유리로 되어있지 않고 따로 구분되어 있어서 좋았어요~\n저희가 더위를 잘타는데 에어'

평가용 함수 작성
- 모델 기반 평가 (Pairwise Comparison)

In [26]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion

Baseline 모델 개발

In [ ]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [25]:
reviews, _ = preprocess_reviews(path='./res/yanolja_review.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'): # 매번 쓰지 않아도 되도록
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

위치가 좋고 시설이 깔끔하며 친절한 직원들이 있어서 편안한 숙박이었어요. 에어컨이 시원해서 푹 잘 수 있었고, 다음에도 또 방문하고 싶은 곳이에요. 근처에 다른 볼거리가 많아서 더욱 즐거웠고, 청결도와 서비스 모두 만족스러웠어요. 전망이 좋아서 짧은 시간이었지만 편안하게 쉬다 갈 수 있었어요.


대조군이 필요한데.... 실제 서비스에서 요약된 결과물 활용

In [23]:
summary_real_20240526 = '위치가 매우 우수한 숙박시설로, 인사동과 조계사, 경복궁 등 관광지에 도보로 이동할 수 있는 편리한 위치에 있습니다. 객실은 깔끔하며 직원들의 친절한 서비스와 청결한 시설이 인상적입니다. 주변에는 맛집과 편의시설이 많아 편리하며, 교통 접근성도 좋습니다. 전체적으로 만족도가 높고 자주 방문하고 싶은 곳으로 손꼽히는 숙소로 평가됩니다.'

In [27]:
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE).choices[0].message.content, summary_real_20240526).choices[0].message.content)


Assistant A와 Assistant B의 답변을 비교해보면 다음과 같습니다:

1. **도움성**: 
   - Assistant A는 리뷰의 주요 포인트를 간략하게 요약하여 제공하고 있습니다. 위치, 시설, 직원의 친절함, 에어컨, 청결도, 서비스, 전망 등 다양한 요소를 언급하여 전반적인 숙박 경험을 잘 전달하고 있습니다.
   - Assistant B는 위치의 우수성, 관광지 접근성, 객실의 깔끔함, 직원의 친절함, 청결한 시설, 주변 맛집과 편의시설, 교통 접근성 등 구체적인 정보를 제공하고 있습니다. 

2. **관련성**:
   - Assistant A는 리뷰의 주요 내용을 충실히 반영하고 있으며, 각 리뷰에서 언급된 요소들을 빠짐없이 포함하고 있습니다.
   - Assistant B는 리뷰에서 언급된 요소들을 잘 반영하고 있으며, 특히 위치와 관련된 정보를 상세히 제공하고 있습니다.

3. **정확성**:
   - Assistant A는 리뷰의 내용을 정확하게 요약하고 있으며, 각 리뷰의 주요 포인트를 잘 반영하고 있습니다.
   - Assistant B도 리뷰의 내용을 정확하게 반영하고 있으며, 특히 위치와 관련된 정보를 구체적으로 제공하고 있습니다.

4. **깊이**:
   - Assistant A는 리뷰의 주요 포인트를 간략하게 요약하고 있어 깊이는 다소 부족할 수 있습니다.
   - Assistant B는 위치와 관련된 정보를 상세히 제공하고 있어 깊이가 더 있습니다.

5. **창의성**:
   - Assistant A는 리뷰의 내용을 충실히 반영하고 있으나, 창의적인 요소는 다소 부족합니다.
   - Assistant B는 위치와 관련된 정보를 구체적으로 제공하며, 창의적인 요소가 더 돋보입니다.

6. **세부사항**:
   - Assistant A는 리뷰의 주요 포인트를 간략하게 요약하고 있어 세부사항이 다소 부족할 수 있습니다.
   - Assistant B는 위치와 관련된 정보를 상세히 제공하고 있어 세부사항이 더 풍부합니다

대규모 평가 스크립트
- 여기서는 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [ ]:
eval_count = 10

# 같은 리뷰에 대해 eval_count만큼 다양하게 요약하기 
summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['나인트리 인사동은 위치가 좋고, 시설이 깔끔하며 서비스가 좋다고 하네요. 침대와 샤워실이 유리로 되어있지 않아서 좋았고, 에어컨이 효과적이어서 편히 잘 수 있었다고 합니다. 다음에 또 방문하고 싶은 곳이라고 하네요. 위치와 청결도, 직원의 친절함 등이 좋다고 하며, 잠도 편안했다고 합니다. 편리한 교통과 아름다운 명소에 가까워 시설, 서비스, 위치 등이 모두 훌륭하다고 하네요.',
 '위치가 좋고 시설이 깔끔하며 에어컨이 시원했던 호텔로, 다시 방문하고 싶다는 의견이 많았습니다. 숙박한 손님들은 넓고 쾌적한 공간에 만족했고, 서비스와 친절함도 좋았다고 언급했습니다. 가격 대비 만족도가 높았으며, 편리한 교통과 아름다운 명소에 근접했다는 점을 강조하며 어메니티와 직원의 친절함을 칭찬했습니다. 이 숙소는 서울 여행을 더욱 특별하게 만들어준다는 긍정적인 리뷰가 많았습니다.',
 '위치가 좋고 시설이 깔끔하며 침대, 샤워실, 화장실이 따로 구분되어 좋았다.\n에어컨이 효과적이어서 푹 자기에 좋았고, 다음에도 오고 싶은 곳이었다.\n다른 볼거리가 주변에 있어 스테이상품 이용했지만 다음에는 숙박으로 놀러가고 싶다.\n서울에 오면 머물고 싶은 호텔이며 위치, 시설, 서비스 모든 것이 완벽했다.\n청결도와 직원 친절도도 좋았고, 전망과 편리한 교통이 더욱 훌륭했다.',
 '위치가 좋고 시설이 깔끔하며 에어컨이 좋아서 편안했고, 다음에도 방문하고 싶은 장소라고 말했습니다. 또 다른 손님은 깔끔하고 넓어서 더 쾌적했다며 다음에는 숙박으로 방문하고 싶다고 했습니다. 세 번째 손님은 나인트리 인사동을 항상 머물고 싶어한다고 말했고 네 번째는 방음도 잘되고 어메니티가 훌륭하다고 칭찬했습니다. 마지막으로 위치가 좋고 청결도, 서비스, 직원의 친절함 등 모든 것이 완벽하다고 칭찬했습니다.',
 '위치와 시설이 좋았고, 침대와 샤워실이 유리로 되어있지 않아서 좋았다. 에어컨이 빵빵해서 편안하게 휴식을 취할 수 있었고, 근처에 다른 볼일이 있어서 스테이상품을 이용했지만 숙박으로 다시 오

In [ ]:
from tqdm import tqdm


def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content # 판정

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else:
            print('Evaluation Error')

    return a_cnt, b_cnt, draw_cnt

wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20240526 for _ in range(len(summaries_baseline))]) # B는 그냥 10개 그대로
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')